## Install Dependencies

In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install redis
!{sys.executable} -m pip install themoviedb
!{sys.executable} -m pip install awscli
!{sys.executable} -m pip install boto3
!{sys.executable} -m pip install botocore
!{sys.executable} -m pip install themoviedb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.0/157.0 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.3 MB/s eta 0:00:00:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2

## Import Libraries

In [2]:
import pandas as pd
import numpy as np
import uuid
#from sentence_transformers import SentenceTransformer
import datetime
import os
import redis
import time
import json
import boto3
import botocore
import pickle
import re
import gzip
import shutil
from urllib import request
from redis.commands.search.field import VectorField
from redis.commands.search.field import TextField, NumericField
from redis.commands.search.field import TagField
from redis.commands.search.query import Query, NumericFilter
from redis.commands.search.result import Result
from redis.cluster import RedisCluster as MemoryDB
from redis.commands.search.field import VectorField, TextField, NumericField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from themoviedb import TMDb
from multiprocessing.pool import ThreadPool

## Test Connection to MemoryDB

In [3]:
%%time
memorydb_host = os.environ.get("MEMORYDB_HOST", "clustercfg.xxxx.xxxxxx.memorydb.xx-xxxx-x.amazonaws.com")
memorydb_port = os.environ.get("MEMORYDB_PORT", 6379)
print(f"MemoryDB Url = {memorydb_host}:{memorydb_port}")
rc = MemoryDB(host=memorydb_host, port=memorydb_port, ssl=True, decode_responses=False, ssl_cert_reqs="none")
rc.ping()
# rc.flushall()

MemoryDB Url = clustercfg.xxxx.xxxxxx.memorydb.xx-xxxx-x.amazonaws.com:6379
CPU times: user 71 ms, sys: 5.03 ms, total: 76 ms
Wall time: 85.9 ms


True

## Download IMDB Datasets

### Define Functions

In [4]:
dataset_path = "datasets"
def download_compressed_dataset(url):
    if not os.path.isdir(dataset_path):
        os.makedirs(dataset_path) 
    compressed_filename = dataset_path + "/" + re.split(pattern='/', string=url)[-1]
    request.urlretrieve(url=url, filename=compressed_filename)
    uncompressed_filename = re.split(pattern=r'\.gz', string=compressed_filename)[0]
    print(f"downloaded {compressed_filename}")
    with gzip.open(compressed_filename, 'rb') as f_in:
        with open(uncompressed_filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print(f"extracted {uncompressed_filename}")
    return uncompressed_filename

In [5]:
datasets = [
    "https://datasets.imdbws.com/title.basics.tsv.gz",
    "https://datasets.imdbws.com/title.principals.tsv.gz",
    "https://datasets.imdbws.com/name.basics.tsv.gz",
    "https://datasets.imdbws.com/title.ratings.tsv.gz",
]

retrieved_datasets = ThreadPool(5).imap_unordered(download_compressed_dataset, datasets)
for retrieved_dataset in retrieved_datasets:
    print(f"loading {retrieved_dataset}")
    pickle.dump(pd.read_table(retrieved_dataset,sep="\t",low_memory=False, na_values=["\\N","nan"]), open(retrieved_dataset[:-4]+".sav","wb"))

downloaded datasets/title.ratings.tsv.gz
extracted datasets/title.ratings.tsv
datasets/title.ratings.tsv
downloaded datasets/title.basics.tsv.gz
downloaded datasets/name.basics.tsv.gz
downloaded datasets/title.principals.tsv.gz
extracted datasets/name.basics.tsv
datasets/name.basics.tsv
extracted datasets/title.basics.tsv
extracted datasets/title.principals.tsv
datasets/title.basics.tsv
datasets/title.principals.tsv


## Show Datasets 

In [17]:
df_title_basics = pickle.load(open(f"{dataset_path}/title.basics.sav","rb"))
df_title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1,"Comedy,Short"


In [7]:
df_title_principals = pickle.load(open(f"{dataset_path}/title.principals.sav","rb"))
df_title_principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,NaN,"[""Self""]"
1,tt0000001,2,nm0005690,director,NaN,NaN
2,tt0000001,3,nm0005690,producer,producer,NaN
3,tt0000001,4,nm0374658,cinematographer,director of photography,NaN
4,tt0000002,1,nm0721526,director,NaN,NaN


In [8]:
df_name_basics = pickle.load(open(f"{dataset_path}/name.basics.sav","rb"))
df_name_basics.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"


In [9]:
df_title_ratings = pickle.load(open(f"{dataset_path}/title.ratings.sav","rb"))
df_title_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2060
1,tt0000002,5.6,278
2,tt0000003,6.5,2029
3,tt0000004,5.4,179
4,tt0000005,6.2,2796


In [45]:
df_title_crew = pickle.load(open(f"{dataset_path}/title.crew.sav","rb"))
df_title_crew.head()

,tconst,directors,writers
0,tt0000001,nm0005690,NaN
1,tt0000002,nm0721526,NaN
2,tt0000003,nm0721526,NaN
3,tt0000004,nm0721526,NaN
4,tt0000005,nm0005690,NaN


## Drop Specific Titles

### Drop Specific Title Types

In [18]:
# initial title count
df_title_basics.shape[0]

10900927

In [19]:
# identify title types
df_title_basics.titleType.unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [20]:
# only keep movie tvMovie tvSeries tvMiniSeries
df_title_basics = df_title_basics[(df_title_basics.titleType == "movie") | (df_title_basics.titleType == "tvMovie") | (df_title_basics.titleType == "tvSeries") | (df_title_basics.titleType == "tvMiniSeries")]
df_title_basics.shape[0]

1154692

### Drop Older Titles

In [21]:
# drop titles older than 1925
df_title_basics = df_title_basics[df_title_basics['startYear'] > 1935]
# df_title_basics = df_title_basics[df_title_basics['endYear'] > 1935]
df_title_basics.shape[0]

977658

### Drop Unused Columns

In [22]:
# drop unused columns
df_title_basics = df_title_basics.drop(['originalTitle', 'endYear'], axis=1)
df_title_basics.columns = ['tconst', 'titleType', 'title', 'isAdult', 'year', 'runtime', 'genres']
df_title_basics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 977658 entries, 3816 to 10900877
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   tconst     977658 non-null  object 
 1   titleType  977658 non-null  object 
 2   title      977656 non-null  object 
 3   isAdult    977658 non-null  float64
 4   year       977658 non-null  float64
 5   runtime    624032 non-null  object 
 6   genres     888267 non-null  object 
dtypes: float64(2), object(5)
memory usage: 59.7+ MB


## Merge Tables

### Prepare Directors Table

In [23]:
df_title_directors = df_title_principals[df_title_principals['category'] == 'director'][['tconst', 'nconst']]
df_title_directors = df_title_directors.merge(df_name_basics[['nconst', 'primaryName']], on='nconst', how='left')
df_title_directors['primaryName'] = df_title_directors['primaryName'].fillna('')
df_title_directors = df_title_directors.groupby('tconst')['primaryName'].apply(lambda x: ', '.join(x)).reset_index()
df_title_directors.columns = ['tconst', 'directors']

### Prepare Actors Table

In [24]:
df_title_actors = df_title_principals[df_title_principals['category'].isin(['actor', 'actress'])][['tconst', 'nconst']]
df_title_actors = df_title_actors.merge(df_name_basics[['nconst', 'primaryName']], on='nconst', how='left')
df_title_actors['primaryName'] = df_title_actors['primaryName'].fillna('')
df_title_actors = df_title_actors.groupby('tconst')['primaryName'].apply(lambda x: ', '.join(x)).reset_index()
df_title_actors.columns = ['tconst', 'actors']

### Merge Tables

In [25]:
df_title_ratings.columns = ['tconst', 'rating', 'numVotes']

df_merged = pd.merge(df_title_basics, df_title_ratings, on='tconst', how='left')
df_merged = pd.merge(df_merged, df_title_directors, on='tconst', how='left')
df_merged = pd.merge(df_merged, df_title_actors, on='tconst', how='left')
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977658 entries, 0 to 977657
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   tconst     977658 non-null  object 
 1   titleType  977658 non-null  object 
 2   title      977656 non-null  object 
 3   isAdult    977658 non-null  float64
 4   year       977658 non-null  float64
 5   runtime    624032 non-null  object 
 6   genres     888267 non-null  object 
 7   rating     468633 non-null  float64
 8   numVotes   468633 non-null  float64
 9   directors  631876 non-null  object 
 10  actors     700546 non-null  object 
dtypes: float64(4), object(7)
memory usage: 82.0+ MB


### Drop More Titles (without directors and without actors)

In [26]:
# drop rows with no directors or actors
df_merged.dropna(subset=['directors'], inplace=True)
df_merged.dropna(subset=['actors'], inplace=True)
df_merged.shape[0]

495262

### Drop Lower Rated Titles (and titles with low number of ratings)

In [27]:
df_merged = df_merged[df_merged['numVotes'] > 2000]
df_merged = df_merged[df_merged['rating'] > 4.0]
df_merged.shape[0]

27880

### Get Plot from TMDB

#### Define Function to Get Plot

In [28]:
from time import sleep
tmdb = TMDb(key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx", language="en-US", region="US")
def get_overview(imdb_id):
    title =  tmdb.find().by_imdb(imdb_id)
    sleep(0.025)
    if title.movie_results:
        return title.movie_results[0].overview
    if  title.tv_results:
        return title.tv_results[0].overview
    return

#### Get Plot for All Remaining Titles

In [29]:
df_subset = df_merged.head(100)
df_subset['plot'] = df_subset['tconst'].map(get_overview)
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 106 to 2201
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tconst     100 non-null    object 
 1   titleType  100 non-null    object 
 2   title      100 non-null    object 
 3   isAdult    100 non-null    float64
 4   year       100 non-null    float64
 5   runtime    100 non-null    object 
 6   genres     100 non-null    object 
 7   rating     100 non-null    float64
 8   numVotes   100 non-null    float64
 9   directors  100 non-null    object 
 10  actors     100 non-null    object 
 11  plot       100 non-null    object 
dtypes: float64(4), object(8)
memory usage: 10.2+ KB


/tmp/ipykernel_8679/1649889164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['plot'] = df_subset['tconst'].map(get_overview)


#### Drop Titles with No Plot

In [30]:
df_subset['plot'].replace('', np.nan, inplace=True)
df_subset.dropna(subset=['plot'], inplace=True)
df_subset.shape[0]

/tmp/ipykernel_8679/2992180993.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_subset['plot'].replace('', np.nan, inplace=True)
/tmp/ipykernel_8679/2992180993.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['plot'].replace('', np.nan, inplace=True)
/tmp/ipykernel_8679/2992180993.py:2: SettingWithCopyWarning: 
A value is tr

100

### Generate Vector Embedding for Plot

#### Define Function to Query Bedrock for Vector Embedding

In [31]:
bedrock_runtime = boto3.client('bedrock-runtime')

def get_text_embedding(text, dimensions=512, normalize=True):
    if not isinstance(text, str):
        return ""
    try:
        body = json.dumps({"inputText": text, "dimensions" : dimensions, "normalize": normalize})
        modelId = "amazon.titan-embed-text-v2:0"
        accept = "application/json"
        contentType = "application/json"

        response = bedrock_runtime.invoke_model(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
        response_body = json.loads(response.get("body").read())
        return(response_body.get("embedding"))
    except botocore.exceptions.ClientError as error:
        print(text)
        if error.response['Error']['Code'] == 'AccessDeniedException':
               print(f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubeshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

        else:
            print(text)
            raise error

#### Add Vector Embedding to Each Title

In [32]:
df_subset["v_plot"] = df_subset["plot"].map(get_text_embedding)
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 106 to 2201
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tconst     100 non-null    object 
 1   titleType  100 non-null    object 
 2   title      100 non-null    object 
 3   isAdult    100 non-null    float64
 4   year       100 non-null    float64
 5   runtime    100 non-null    object 
 6   genres     100 non-null    object 
 7   rating     100 non-null    float64
 8   numVotes   100 non-null    float64
 9   directors  100 non-null    object 
 10  actors     100 non-null    object 
 11  plot       100 non-null    object 
 12  v_plot     100 non-null    object 
dtypes: float64(4), object(9)
memory usage: 10.9+ KB


/tmp/ipykernel_8679/1171507248.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset["v_plot"] = df_subset["plot"].map(get_text_embedding)


In [33]:
df_subset.head()

,tconst,titleType,title,isAdult,year,runtime,genres,rating,numVotes,directors,actors,plot,v_plot
106,tt0027260,movie,After the Thin Man,0.0,1936.0,112,"Comedy,Crime,Mystery",7.6,12204.0,W.S. Van Dyke,"William Powell, Myrna Loy, James Stewart, Elis...",Nick and Nora Charles investigate when Nora's ...,"[-0.112098925, 0.0368897, -0.00024240141, -0.0..."
121,tt0027286,movie,The Amazing Adventure,0.0,1936.0,61,"Comedy,Drama,Romance",6.3,2191.0,Alfred Zeisler,"Cary Grant, Mary Brian, Peter Gawthorne, Henry...",A bored millionaire wagers his doctor that he ...,"[-0.11646503, -0.016336221, 0.022670673, -0.11..."
134,tt0027300,movie,Anthony Adverse,0.0,1936.0,141,"Adventure,Drama,Romance",6.3,2046.0,Mervyn LeRoy,"Fredric March, Olivia de Havilland, Donald Woo...","Based on the novel by Hervey Allen, this expan...","[-0.10065765, 0.01913084, -0.029284902, 0.0194..."
158,tt0027336,movie,The Lower Depths,0.0,1936.0,90,"Crime,Drama,Romance",7.5,3701.0,Jean Renoir,"Jean Gabin, Suzy Prim, Louis Jouvet, Jany Holt...",Inhabitants of a flophouse struggle to survive...,"[-0.049684554, 0.120132804, 0.0017727931, -0.0..."
180,tt0027367,movie,Black Legion,0.0,1937.0,83,"Crime,Drama,Film-Noir",6.9,3940.0,Archie Mayo,"Humphrey Bogart, Ann Sheridan, Dick Foran, Eri...",When a hard-working machinist loses a promotio...,"[-0.09975542, 0.11154916, 0.009889546, -0.0111..."


## MemoryDB Ingestion

### Define Functions for Index Creation

In [34]:
def generate_key(prefix = ""):
    return prefix + str(uuid.uuid4())

def create_hnsw_index(rc, index_name, vector_field_name, number_of_vectors, vector_dimensions=512, distance_metric='L2', M=16, EF=512, key_prefix=''):
    # Create the index
    try:
        rc.ft(index_name).create_index([
            VectorField(vector_field_name, "HNSW", {
                "TYPE": "FLOAT32",
                "DIM": vector_dimensions,
                "DISTANCE_METRIC": distance_metric,
                "INITIAL_CAP": number_of_vectors,
                "M": M,
                "EF_CONSTRUCTION": EF
            }),
            TagField("isAdult"),
            TagField("titleType"),
            TagField("directors"),
            NumericField("rating"),
            TagField("genres"),
            TextField("plot"),
            TextField("title"),
            NumericField("runtime"),
            TagField("actors"),
            NumericField("year")
        ], definition=IndexDefinition(prefix=[key_prefix]))
        print(f"Index {index_name} created successfully.")
    except ResponseError as e:
        print(f"Index {index_name} created previously: {str(e)}")

### Define Variables for MemoryDB Index

In [35]:
KEY_PREFIX = "movies:"
index_name = "imdb_hnsw_index"
vector_field_name = "v_plot"
number_of_vectors = len(df_subset)
print(f"Creating Index {index_name} on Field {vector_field_name} expecting {number_of_vectors:,} vectors")

Creating Index imdb_hnsw_index on Field v_plot expecting 100 vectors


### Create Index (and delete existing items)

In [36]:
%%time
# Create index in MemoryDB
rc.ft(index_name).dropindex()
for key in rc.scan_iter("movies:*"):
    rc.delete(key)
create_hnsw_index(rc, index_name, vector_field_name, number_of_vectors, 
                  vector_dimensions=512, distance_metric='Cosine', M=16, EF=512, key_prefix=KEY_PREFIX)

Index imdb_hnsw_index created successfully.
CPU times: user 2.81 s, sys: 492 ms, total: 3.3 s
Wall time: 1min 35s


### Load Data into MemoryDB

In [37]:
%%time
# Load data into MemoryDB
BATCH_SIZE = 100
pipe = rc.pipeline()
for index, row in df_subset.iterrows():
    key = generate_key(prefix=KEY_PREFIX)
    v_plot = np.array(row['v_plot'], dtype=np.float32).tobytes()
    pipe.hset(key, mapping={
        'directors': row['directors'],
        'rating': row['rating'],
        'genres': row['genres'],
        'plot': row['plot'],
        'title': row['title'],
        'runtime': row['runtime'],
        'actors': row['actors'],
        'year': row['year'],
        'titleType': row['titleType'],
        'isAdult': row['isAdult'],
        'v_plot': v_plot
    })
    if index % BATCH_SIZE == 0 or index == number_of_vectors - 1:
        pipe.execute()
        pipe = rc.pipeline()
print("Data indexed successfully.")

Data indexed successfully.
CPU times: user 11.3 ms, sys: 7.11 ms, total: 18.4 ms
Wall time: 25.9 ms


### Find Random Key and Fetch Value

In [38]:
%%time
# Add a python script to find a random key that stats with the prefix and fetch the value and show it
count = 0
while True:
    count += 1
    keyname = rc.randomkey()
    keyname = keyname.decode('utf-8')
    print(str(keyname))
    if keyname.startswith(KEY_PREFIX) == True:
        print(rc.hgetall(keyname))
        break
    elif count > 10:
        break

movies:6ebce7d8-0882-43d4-8d7b-876ada1a2420
{b'actors': b'Warner Oland, Boris Karloff, Keye Luke, Charlotte Henry, Thomas Beck, Margaret Irving, Gregory Gaye, Nedda Harrigan, Frank Conroy, Guy Usher', b'titleType': b'movie', b'title': b'Charlie Chan at the Opera', b'rating': b'7.1', b'year': b'1936.0', b'plot': b'A dangerous amnesiac escapes from an asylum, hides in the opera house, and is suspected of getting revenge on those who tried to murder him 13 years ago.', b'isAdult': b'0.0', b'runtime': b'68', b'genres': b'Mystery', b'v_plot': b' \xf0\x85\xbd\x08\xeb0>R\xea6=;\xee\x95\xbdp\xea\xb5\xbd\xa3\xed\x1a=-\xef\r\xbcC\xe2\xfa\xbcH\xed\x1d=%\xe2{=\x11\xe8\xc9<\xc1\xed\x99;\xad\xea\xb3=-\xef\x8d=\x9c\xe9<=\xc4\xe6T\xbc\xed\xec \xbdC\xe2\xfa\xbc\xbd\xe2\xf6=\xa3\xed\x9a\xbc\xdc\xeb\xa9=\xe0\xed\x98<\xed\xec \xbd\xd2\xee\x90\xbb\x8e\xea\xb4\xbd\xc1\xed\x99<\xe9\xea\xb1\xbc\x80\xe2x:C\xe2\xfa\xbc>\xf0\x84<\x1b\xe5\xe2<\xb4\xee\x91\xbc7\xec&\xbdA\xe9?\xbd\xef\xe5\xdb\xbc\xc1\xed\x99=\n\xe4

### Query MemoryDB

#### Define Functions

In [39]:
def similarity_search(redis_client, index_name, query_vector, filters="*",top_n=1):
    # Convert the query vector to bytes
    query_vector_bytes = np.array(query_vector, dtype=np.float32).tobytes()
    # Create the query
    query = Query(f"{filters}=>[KNN {top_n} @v_plot  $query_vec AS score ]") \
        .sort_by("score") \
        .return_fields(  "titleType", "title", "year", "plot", "rating", "genres", "runtime") \
        .paging(0, top_n) \
        .dialect(2)
    params = {
        "query_vec": query_vector_bytes
    }
    # Process the query
    result = redis_client.ft(index_name).search(query, query_params=params)
    return result
def format_results(results):
    for doc in results.docs:
        doc_id = doc.id
        titleType = doc.titleType
        title = doc.title
        year = doc.year
        rating = doc.rating
        genres = doc.genres
        runtime = doc.runtime
        print(f"ID: {doc_id}, Title Type: {titleType}, Title: {title}, Year: {year}, Rating: {rating}, Genres: {genres}, Runtime: {str(datetime.timedelta(seconds=float(runtime)))}")

#### Get Random Title from Dataset

In [40]:
randomdf = df_subset.sample()
randomdf.get("title")

1443    History Is Made at Night
Name: title, dtype: object

In [41]:
selected_row = randomdf.iloc[0]
print(selected_row)
query_vector = selected_row['v_plot']

tconst                                               tt0029002
titleType                                                movie
title                                 History Is Made at Night
isAdult                                                    0.0
year                                                    1937.0
runtime                                                     97
genres                                    Comedy,Drama,Romance
rating                                                     7.3
numVotes                                                2565.0
directors                                        Frank Borzage
actors       Charles Boyer, Jean Arthur, Leo Carrillo, Coli...
plot         An American woman falls in love with a romanti...
v_plot       [-0.10615807, 0.0035840196, -0.06018783, -0.08...
Name: 1443, dtype: object


#### Do Simple Similarity Search

In [42]:
%%time
results = similarity_search(rc, index_name, query_vector, top_n=15)

CPU times: user 1.26 ms, sys: 131 µs, total: 1.39 ms
Wall time: 1.99 ms


In [43]:
format_results(results)

ID: movies:cd6bcb0d-a60b-4efa-ad11-895ae63f7af4, Title Type: movie, Title: History Is Made at Night, Year: 1937, Rating: 7.3, Genres: Comedy,Drama,Romance, Runtime: 0:01:37
ID: movies:89e1ccd2-5f3a-4e95-97e2-24fc1624a985, Title Type: movie, Title: Angel, Year: 1937, Rating: 7.2, Genres: Comedy,Drama,Romance, Runtime: 0:01:31
ID: movies:74d27feb-b1d4-4d90-a121-b0aea163050e, Title Type: movie, Title: Camille, Year: 1936, Rating: 7.3, Genres: Drama,Romance, Runtime: 0:01:49
ID: movies:46f4ff91-c88a-4f4f-9d0b-19f7c7487a4c, Title Type: movie, Title: Double Wedding, Year: 1937, Rating: 6.9, Genres: Comedy,Romance, Runtime: 0:01:27
ID: movies:d420fd5a-7b20-4554-aa58-4c0eb1333a51, Title Type: movie, Title: Desire, Year: 1936, Rating: 7.1, Genres: Comedy,Crime,Romance, Runtime: 0:01:35
ID: movies:eb334af3-8b38-4c82-bbae-cbc1d767af1f, Title Type: movie, Title: Marked Woman, Year: 1937, Rating: 7.1, Genres: Crime,Film-Noir,Thriller, Runtime: 0:01:36
ID: movies:b4af7f8e-222f-4cb5-bc88-52427bc781fb

#### Do Similarity Search with Filters

In [44]:
%%time
filters = "@titleType:{movie} @year:[2010 2020] @rating:[6 (INF] (@genres:{Horror} | @genres:{Animation} | @genres:{Action}| @genres:{Drama}) "
results = similarity_search(rc, index_name, query_vector, filters=filters, top_n=15)

CPU times: user 744 µs, sys: 77 µs, total: 821 µs
Wall time: 2.01 ms


In [45]:
format_results(results)

## Save Dataset

In [46]:
pickle.dump(df_subset, open(f"{dataset_path}/title.subset.sav","wb"))